In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from logger import Logger

# Suppressing tf.hub warnings
tf.get_logger().setLevel("ERROR")
print(tf.__version__)
print(tf.__version__, tf.config.list_physical_devices("GPU"))


2.11.0
2.11.0 [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
def load_tf_mnist():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
    x_test = x_test.astype('float32')
    x_train = (x_train / 127.5) - 1 
    x_test = (x_test / 127.5) - 1
    
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(60000).batch(64)
    #test_dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

    return train_dataset, x_test, y_test #, test_dataset

In [5]:
def load_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
    x_test = x_test.astype('float32')
    x_train = (x_train / 127.5) - 1 
    x_test = (x_test / 127.5) - 1

    data = np.load('/home/jkopcan/s/hdd/Inverse_Transformation/hypersphere_data.npz')
    anomaly_train = data['anomaly_train']
    original_train = data['original_train']
    train = tf.concat([anomaly_train, original_train], axis=0)
    
    train_dataset = tf.data.Dataset.from_tensor_slices((train,y_train)).shuffle(60000).batch(64)

    return train_dataset

In [3]:
train_dataset, imgs, targets = load_tf_mnist() #load_data()

In [11]:
def label_condition_disc(in_shape=(28, 28, 1), n_classes=10, embedding_dim=100):  #100
    # label input
    con_label = layers.Input(shape=(1,))
    # embedding for categorical input
    label_embedding = layers.Embedding(n_classes, embedding_dim)(con_label)
    # scale up to image dimensions with linear activation
    nodes = in_shape[0] * in_shape[1] * in_shape[2]
    label_dense = layers.Dense(nodes)(label_embedding)
    # reshape to additional channel
    label_reshape_layer = layers.Reshape((in_shape[0], in_shape[1], 1))(label_dense)
    # image input
    return con_label, label_reshape_layer


def image_disc(in_shape=(28,28, 1)):
    inp_image = layers.Input(shape=in_shape)
    return inp_image

def get_base_degenerator_model():
    con_label, label_condition_output = label_condition_disc()
    inp_image_output = image_disc()
    # concat label as a channel
    merge = layers.Concatenate()([inp_image_output, label_condition_output])

    # 28x28x128 --> base model
    x = tf.keras.layers.Conv2D(128, (4,4), padding='same')(merge)
    x = layers.LeakyReLU(alpha=0.2, name="model28x28")(x)
    #x = layers.BatchNormalization()(x)
    assert x.shape == (None, 28, 28, 128)

    # 14x14x128 --> 14x14 model
    x = tf.keras.layers.Conv2D(128, (4,4), strides=(2,2), padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2, name="model14x14")(x)
    #x = layers.BatchNormalization()(x)
    assert x.shape == (None, 14, 14, 128)

    # 7x7x128 --> 7x7 model
    x = tf.keras.layers.Conv2D(128, (2,2), strides=(2,2), padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2, name="model7x7")(x)
    #x = layers.BatchNormalization()(x)
    assert x.shape == (None, 7, 7, 128)

    # batch_sizex6272 --> dense model
    flattened_out = tf.keras.layers.Flatten(name="model_dense")(x)
    assert flattened_out.shape == (None, 7*7*128)
    
    output = tf.keras.layers.Dense(100)(flattened_out)

    model = tf.keras.Model([inp_image_output, con_label], output)  
    return model

In [ ]:
gen = tf.keras.models.load_model("/home/jkopcan/s/hdd/Inverse_Transformation/saved_models/GANs/cGenerator_64BatchSize_v2.h5")
gen_28 = tf.keras.models.Model(inputs=gen.input, outputs=gen.get_layer('layer28x28').output)
gen_14 = tf.keras.models.Model(inputs=gen.input, outputs=gen.get_layer('layer14x14').output)

noise = tf.random.normal([100, 100])
#target = np.random.randint(0, 10, size=100)
#target = tf.convert_to_tensor(target)
#target = tf.reshape(target, (100, 1))

generated_images_28 = gen_28([noise, targets[:100]], training=False)
generated_images_28 = tf.math.reduce_mean(generated_images_28, axis=-1)

generated_images_14 = gen_14([noise, targets[:100]], training=False)
generated_images_14 = tf.math.reduce_mean(generated_images_14, axis=-1)

#np.savez("generated_images_from_gan", images_14x14=generated_images_14, images_28x28=generated_images_28)

from matplotlib import gridspec
output = generated_images_14
k = 0
nrow = 10
ncol = 10
fig = plt.figure(figsize=(15,15))
gs = gridspec.GridSpec(nrow, ncol, width_ratios=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
     wspace=0.0, hspace=0.0, top=0.95, bottom=0.05, left=0.17, right=0.845) 


for i in range(10):
    for j in range(10):
        pred = (output[k, :, :] + 1 ) * 127.5
        ax= plt.subplot(gs[i,j])
        pred = np.array(pred)
        ax.imshow(pred.astype(np.uint8), cmap='gray')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.axis('off')
        k += 1   


plt.savefig(f"images/gen_14x14im.png",  dpi=300)
plt.show()


In [ ]:
#degenerator = get_base_degenerator_model()
degenerator = tf.keras.models.load_model("/home/jkopcan/s/hdd/Inverse_Transformation/saved_models/inverse_trans_model/InverseTrans_64BatchSize_2700e_MSE.h5")
imgs = gen([noise, targets[:100]], training=False)

degenerator_28 = tf.keras.models.Model(inputs=degenerator.input, outputs=degenerator.get_layer('model28x28').output)
degenerator_14 = tf.keras.models.Model(inputs=degenerator.input, outputs=degenerator.get_layer('model14x14').output)

inverse_imgs_28x28 = degenerator_28([imgs, targets[:100]], training=False)
inverse_imgs_14x14 = degenerator_14([imgs, targets[:100]], training=False)

inverse_imgs_28x28 = tf.math.reduce_mean(inverse_imgs_28x28, axis=-1)
inverse_imgs_14x14 = tf.math.reduce_mean(inverse_imgs_14x14, axis=-1)

output = inverse_imgs_14x14
k = 0
nrow = 10
ncol = 10
fig = plt.figure(figsize=(15,15))
gs = gridspec.GridSpec(nrow, ncol, width_ratios=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
     wspace=0.0, hspace=0.0, top=0.95, bottom=0.05, left=0.17, right=0.845) 


for i in range(10):
    for j in range(10):
        pred = (output[k, :, :] + 1 ) * 127.5
        ax= plt.subplot(gs[i,j])
        pred = np.array(pred)
        ax.imshow(pred.astype(np.uint8), cmap='gray')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.axis('off')
        k += 1   


plt.savefig(f"images/inverseT_14x14im.png",  dpi=300)
plt.show()

In [ ]:
gen = tf.keras.models.load_model("/home/jkopcan/s/hdd/Inverse_Transformation/saved_models/GANs/cGenerator_64BatchSize_v2.h5")
gen.summary()
noise = tf.random.normal([1, 100])
target = tf.ones([1, 1])
generated_image = gen([noise, target], training=False)

degenerator = get_base_degenerator_model()
generator28 = tf.keras.models.Model(inputs=gen.input, outputs=gen.get_layer('layer28x28').output)

degenerator_28 = tf.keras.models.Model(inputs=degenerator.input, outputs=degenerator.get_layer('model28x28').output)

output = generator28.predict([noise,target])
print(output.shape)
plt.imshow(output[0, :, :, 0], cmap='gray')

In [14]:
class InverseTransformation(tf.keras.Model):
    def __init__(
        self,
        degenerator,
        generator,
        latent_dim,
    ):
        super(InverseTransformation, self).__init__()
        self.degenerator = degenerator
        self.generator = generator
        self.latent_dim = latent_dim
        self.g_layers_model = tf.keras.models.Model(inputs=generator.input, 
                                             outputs=[generator.get_layer('layer28x28').output, 
                                                      generator.get_layer('layer14x14').output, 
                                                      generator.get_layer('layer7x7').output])
        self.d_layers_model = tf.keras.models.Model(inputs=degenerator.input, 
                                             outputs=[degenerator.get_layer('model28x28').output, 
                                                      degenerator.get_layer('model14x14').output, 
                                                      degenerator.get_layer('model7x7').output])

    def compile(self, d_optimizer, loss_fn):
        super(InverseTransformation, self).compile()
        self.d_optimizer = d_optimizer
        self.loss_fn = loss_fn
        self.mse_metric = tf.keras.metrics.MeanSquaredError(name='mse')
        
    def train_step(self, data):  #data

        # if isinstance(data, tuple):
        #   real_images = data[0]
        #   target = data[1]
        #real_images, target = data
        noise, labels = data

        # sample random noise for generator
        batch_size = tf.shape(noise)[0]

        #random_latent_positive =  tf.random.uniform([(batch_size//2), self.latent_dim],minval=1,maxval=2) 
        #random_latent_negative = tf.random.uniform([(batch_size//2), self.latent_dim],minval=(2*(-1)), maxval=(1*(-1)))
        #random_latent_anomalies = tf.concat([random_latent_positive, random_latent_negative], axis=0)

        #random_latent_vectors =  tf.random.uniform([batch_size, self.latent_dim],minval=(tf.sqrt(2/3)*(-1)), maxval=tf.sqrt(2/3))

        #noise = tf.concat( [random_latent_anomalies, random_latent_vectors], axis=0)
        #labels = target #tf.concat([target, target], axis=0)

        with tf.GradientTape() as d_tape:
            generated_images = self.generator([noise,labels], training=False)
            conv_28, conv_14, conv_7 = self.g_layers_model([noise, labels])

            predictions = self.degenerator([generated_images,labels], training=True)
            deconv_28, deconv_14, deconv_7 = self.d_layers_model([generated_images,labels])
            
            conv_28_loss = self.loss_fn(conv_28, deconv_28)
            conv_14_loss = self.loss_fn(conv_14, deconv_14)
            conv_7_loss = self.loss_fn(conv_7, deconv_7)
            loss = self.loss_fn(noise, predictions)
            d_loss = loss + conv_28_loss + conv_14_loss + conv_7_loss

            self.mse_metric.update_state(noise, predictions)
            
        # gradient calculation for discriminator for real labels    
        gradients_of_d = d_tape.gradient(d_loss, self.degenerator.trainable_variables)
        
        # parameters optimization for discriminator for real labels   
        self.d_optimizer.apply_gradients(zip(gradients_of_d, self.degenerator.trainable_variables))

        # self.logger.log(f"Conv 28x28 Transformation Loss: {conv_28_loss.numpy()}")
        # self.logger.log(f"Conv 14x14 Transformation Loss: {conv_14_loss}")
        # self.logger.log(f"Conv 7x7 Transformation Loss: {conv_7_loss}")
        # self.logger.log(f"Combined Transformation Loss: {d_loss}")
        return {"d_loss": d_loss, "mse": self.mse_metric.result()}


def get_inverse_transformation_model(compile=True):
    latent_dim = 100

    degen = get_base_degenerator_model()

    gen = tf.keras.models.load_model("/home/jkopcan/s/hdd/Inverse_Transformation/saved_models/GANs/cGenerator_64BatchSize_v2.h5")

    inverse_transformation = InverseTransformation(degenerator=degen, generator=gen, latent_dim=latent_dim)

    if compile:
        inverse_transformation.compile(
            d_optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.99, beta_2=0.999),
            loss_fn=tf.keras.losses.MeanSquaredError()
        )
    return inverse_transformation

In [15]:
train_data= load_data() #load_tf_mnist()
iae = get_inverse_transformation_model(compile=True)

In [47]:
import logging

class LossLoggerCallback(tf.keras.callbacks.Callback):
    def __init__(self, log_file):
        super(LossLoggerCallback, self).__init__()
        self.log_file = log_file

    def on_epoch_end(self, epoch, logs=None):
        logging.basicConfig(filename=self.log_file, level=logging.INFO)
        time= datetime.datetime.now()
        time= time.strftime("%Y-%m-%d %H:%M:%S")
        logging.info(f"{time} Epoch {epoch}: Training Loss = {logs['d_loss']}")

# Define the callback to save the model
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/home/jkopcan/s/hdd/Inverse_Transformation/saved_models/inverse_trans_model/degenerator.h5',  # change the path to your desired save location
    monitor='d_loss',  # monitor the loss
    save_best_only=True,  # save only the best model based on monitored metric
    save_weights_only=False,  # save the entire model, not just the weights
    verbose=1  # print progress updates
)

# Get current datetime
now = datetime.datetime.now()
# Format datetime as string
timestamp_str = now.strftime("%Y-%m-%d_%H-%M-%S")
# Define filename with timestamp
#filename = f'model_{timestamp_str}.h5'

loss_logger = LossLoggerCallback(log_file='training_loss_{timestamp_str}.log')
history = iae.fit(train_data, epochs=2000, callbacks=[loss_logger])

Epoch 1/2000
938/938 [==============================] - 16s 16ms/step - d_loss: 5.8639 - mse: 1.2269
Epoch 2/2000
938/938 [==============================] - 16s 16ms/step - d_loss: 5.8642 - mse: 1.2264
Epoch 3/2000
938/938 [==============================] - 15s 16ms/step - d_loss: 5.8615 - mse: 1.2231
Epoch 4/2000
938/938 [==============================] - 16s 16ms/step - d_loss: 5.8611 - mse: 1.2253
Epoch 5/2000
938/938 [==============================] - 16s 17ms/step - d_loss: 5.8645 - mse: 1.2282
Epoch 6/2000
938/938 [==============================] - 16s 17ms/step - d_loss: 5.8643 - mse: 1.2278
Epoch 7/2000
938/938 [==============================] - 16s 16ms/step - d_loss: 5.8601 - mse: 1.2238
Epoch 8/2000
938/938 [==============================] - 16s 17ms/step - d_loss: 5.8641 - mse: 1.2268
Epoch 9/2000
938/938 [==============================] - 16s 17ms/step - d_loss: 5.8631 - mse: 1.2256
Epoch 10/2000
938/938 [==============================] - 16s 17ms/step - d_loss: 5.8579 - m

In [48]:
def load_val_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    y_test = y_test.astype('float32')
    print(y_test.shape)
    data = np.load('/home/jkopcan/s/hdd/Inverse_Transformation/hypersphere_data.npz')
    anomaly_val = data['anomaly_val']
    original_val = data['original_val']
    val = tf.concat([anomaly_val[:5000], original_val[:5000]], axis=0)
    val_data = tf.data.Dataset.from_tensor_slices((val, y_test)).batch(64)
    return val_data, y_test

def create_imgs(model, dataset, y_test):
    generated_imgs = None
    for data, labels in dataset:
        imgs = model((data, labels), training=False)
        if generated_imgs is None:
            generated_imgs = imgs
        else:
            generated_imgs = tf.concat([generated_imgs, imgs], axis=0)
    img_dataset = tf.data.Dataset.from_tensor_slices((generated_imgs, y_test)).batch(64)
    print(img_dataset)
    return generated_imgs,img_dataset

def reconstruct_noise(model, img_dataset):
    recon_data = None
    for imgs, labels in img_dataset:
        reconstructed = model((imgs,labels), training=False)
        if recon_data is None:
            recon_data = reconstructed
        else:
            recon_data = tf.concat([recon_data, reconstructed], axis=0)

    return recon_data

val_dataset, y_test = load_val_data()
#print(val_dataset, y_test.shape)
generated_imgs, img_dataset = create_imgs(gen, val_dataset, y_test)
reconstructed = reconstruct_noise(iae.degenerator, img_dataset)
print(reconstructed.shape)


(10000,)
<BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>
(10000, 100)


In [49]:
inside_count = np.sum(np.linalg.norm(reconstructed, axis=1) <= 1)
outside_count = np.sum(np.linalg.norm(reconstructed, axis=1) > 1)

print("Number of samples inside the unit hypersphere:", inside_count)
print("Number of samples outside the unit hypersphere:", outside_count)

Number of samples inside the unit hypersphere: 0
Number of samples outside the unit hypersphere: 10000


In [34]:
import logging

class LossLoggerCallback(tf.keras.callbacks.Callback):
    def __init__(self, log_file):
        super(LossLoggerCallback, self).__init__()
        self.log_file = log_file

    def on_epoch_end(self, epoch, logs=None):
        logging.basicConfig(filename=self.log_file, level=logging.INFO)
        logging.info(f"Epoch {epoch}: Training Loss = {logs['d_loss']}")

In [50]:
iae.degenerator.save("/home/jkopcan/s/hdd/Inverse_Transformation/saved_models/inverse_trans_model/InverseTrans_64BatchSize_2700e_MSE.h5")

In [ ]:
class InverseTransformation(tf.keras.Model):
    def __init__(
        self,
        degenerator,
        generator,
        latent_dim,
    ):
        super(InverseTransformation, self).__init__()
        self.degenerator = degenerator
        self.generator = generator
        self.latent_dim = latent_dim
        self.g_layers_model = tf.keras.models.Model(inputs=generator.input, 
                                             outputs=[generator.get_layer('layer28x28').output, 
                                                      generator.get_layer('layer14x14').output, 
                                                      generator.get_layer('layer7x7').output])
        self.d_layers_model = tf.keras.models.Model(inputs=degenerator.input, 
                                             outputs=[degenerator.get_layer('model28x28').output, 
                                                      degenerator.get_layer('model14x14').output, 
                                                      degenerator.get_layer('model7x7').output])

    def compile(self, d_optimizer, loss_fn):
        super(InverseTransformation, self).compile()
        self.d_optimizer = d_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data): 
        
        real_images, target = data

        # sample random noise for generator
        batch_size = tf.shape(real_images)[0]

        random_latent_positive =  tf.random.uniform([(batch_size//2), self.latent_dim],minval=1,maxval=2) 
        random_latent_negative = tf.random.uniform([(batch_size//2), self.latent_dim],minval=(2*(-1)), maxval=(1*(-1)))
        random_latent_anomalies = tf.concat([random_latent_positive, random_latent_negative], axis=0)

        random_latent_vectors =  tf.random.uniform([batch_size, self.latent_dim],minval=(tf.sqrt(2/3)*(-1)), maxval=tf.sqrt(2/3))

        noise = tf.concat( [random_latent_anomalies, random_latent_vectors], axis=0)
        labels = tf.concat([target, target], axis=0)

        with tf.GradientTape() as d_tape:
            generated_images = self.generator([noise,labels], training=False)
            conv_28, conv_14, conv_7 = self.g_layers_model([noise, labels])

            predictions = self.degenerator([generated_images,labels], training=True)
            deconv_28, deconv_14, deconv_7 = self.d_layers_model([generated_images,labels])
            
            conv_28_loss = self.loss_fn(conv_28, deconv_28)
            conv_14_loss = self.loss_fn(conv_14, deconv_14)
            conv_7_loss = self.loss_fn(conv_7, deconv_7)
            loss = self.loss_fn(noise, predictions)
            d_loss = loss + conv_28_loss + conv_14_loss + conv_7_loss
            
        # gradient calculation for discriminator for real labels    
        gradients_of_d = d_tape.gradient(d_loss, self.degenerator.trainable_variables)
        
        # parameters optimization for discriminator for real labels   
        self.d_optimizer.apply_gradients(zip(gradients_of_d, self.degenerator.trainable_variables))

        return {"d_loss": d_loss}